In [1]:
# Dependencies
import requests
import pandas as pd
import datetime

### Data Retrieval

In [2]:
# Query url for HPD recent crime reports (rpt) from COHGIS Open Data Portal
url = "https://opendata.arcgis.com/datasets/8d515a90e80840b3bc7a3ada352b0d15_0.geojson"

# Get HPD crime reports data and save as "hpd_crime_rpt"
hpd_crime_rpt = requests.get(url).json()

# View "hpd_crime_rpt"
hpd_crime_rpt

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'OBJECTID': 2001,
    'Join_Count': 1,
    'TARGET_FID': 2001,
    'Join_Count_1': 1,
    'Incident_No': 33634519,
    'Time_Begun': '2019-03-16T20:45:00.000Z',
    'UCR_No': 7001,
    'HPD_Beat': '10H70',
    'HPD_District': '10',
    'Offense': 'Auto Theft',
    'Address_Range': '2300-2399 CAMDEN',
    'Premise_Type': 'Road, Street, or Sidewalk',
    'X_Coord': 3117097.76176,
    'Y_Coord': 13823911.50133,
    'Zip_Code': '77021',
    'Time_Frame': 2,
    'SNB_No': 83,
    'SNB_Name': 'MACGREGOR',
    'Council_District': 'D',
    'HPD_Division': 'South Central Division'},
   'geometry': {'type': 'Point',
    'coordinates': [-95.38101631427402, 29.711213511623697]}},
  {'type': 'Feature',
   'properties': {'OBJECTID': 2002,
    'Join_Count': 1,
    'TARGET_FID': 2002,
    'Join_Count_1': 1,
    'Incident_No': 27829919,
    'Time_Begun': '2019-03-04T13:25:00.000Z',
    'UCR_No': 7031,
    'HPD_Beat': '12D

In [3]:
# Convert "hpd_crime_report" to Pandas dataframe
hpd_crime_rpt_pd = pd.DataFrame(hpd_crime_rpt)

# View "hpd_crime_rpt_pd"
hpd_crime_rpt_pd.head()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'properties': {'OBJECTID':..."
1,FeatureCollection,"{'type': 'Feature', 'properties': {'OBJECTID':..."
2,FeatureCollection,"{'type': 'Feature', 'properties': {'OBJECTID':..."
3,FeatureCollection,"{'type': 'Feature', 'properties': {'OBJECTID':..."
4,FeatureCollection,"{'type': 'Feature', 'properties': {'OBJECTID':..."


In [4]:
# Write "hpd_crime_rpt_pd" to csv file
hpd_crime_rpt_pd.to_csv("../data/rawdata/hpd_crime_report.csv", index=False, header=True)

### Data Cleaning

###### >> Sort out retrieved crime lists

In [5]:
# Grab "features" data from "hpd_crime_rpt"
crime_list = hpd_crime_rpt["features"]

# Check the length of "crime_list"
print(f"There are {len(crime_list)} crime cases reported in the HPD recent crime report")

# Preview a representative "crime_list" structure
crime_list[0]

There are 7368 crime cases reported in the HPD recent crime report


{'type': 'Feature',
 'properties': {'OBJECTID': 2001,
  'Join_Count': 1,
  'TARGET_FID': 2001,
  'Join_Count_1': 1,
  'Incident_No': 33634519,
  'Time_Begun': '2019-03-16T20:45:00.000Z',
  'UCR_No': 7001,
  'HPD_Beat': '10H70',
  'HPD_District': '10',
  'Offense': 'Auto Theft',
  'Address_Range': '2300-2399 CAMDEN',
  'Premise_Type': 'Road, Street, or Sidewalk',
  'X_Coord': 3117097.76176,
  'Y_Coord': 13823911.50133,
  'Zip_Code': '77021',
  'Time_Frame': 2,
  'SNB_No': 83,
  'SNB_Name': 'MACGREGOR',
  'Council_District': 'D',
  'HPD_Division': 'South Central Division'},
 'geometry': {'type': 'Point',
  'coordinates': [-95.38101631427402, 29.711213511623697]}}

###### >>Categorize types of crimes reported

In [6]:
# list to store types of crimes
crime_type = []

# Loop through "crime_list"
for case in crime_list:
    
    # Append unique values of crime types to "crime_type" list
    if case["properties"]["Offense"] not in crime_type:
        crime_type.append(case["properties"]["Offense"])

# View "crime_type"
crime_type

['Auto Theft',
 'Theft',
 'Burglary',
 'Aggravated Assault',
 'Robbery',
 'Rape',
 'Murder']

In [7]:
# List for violent crimes
violent_crime = ["Aggravated Assault", "Murder", "Robbery", "Rape"]

# List for non-violent crimes
non_violent_crime = ["Theft", "Burglary", "Auto Theft"]

###### >> Slim and clean up crime data

In [8]:
# Generate a new empty Pandas dataframe with columns of "Offense", "Crime Type", "Lat", and "Lng"
crime_info_pd = pd.DataFrame(columns = ["Offense", "Crime Type", "ZIP", "LAT", "LNG"])

# Check "crime_info_pd"
crime_info_pd

,Offense,Crime Type,ZIP,LAT,LNG


In [9]:
# List to store date of crime
crime_date = []

# Loop through "crime_list"
for case in crime_list:

    # Note that adding value to column immediately changes the length of "crime_info_pd"
    # Temporarily assign (fix) it to intermediate variable "im" to be referred before each iteration
    im = len(crime_info_pd)
    
    # Append data to "Offense" column of "crime_info_pd"
    crime_info_pd.loc[im, "Offense"] = case["properties"]["Offense"]
    
    # Append data to "Zip" column of "crime_info_pd"
    crime_info_pd.loc[im, "ZIP"] = case["properties"]["Zip_Code"]
    
    # Determine crime type (violent/non-violent) via comparison with "violent_crime" and "non_violent_crime" lists
    if case["properties"]["Offense"] in violent_crime:
        crime_info_pd.loc[im, "Crime Type"] = "Violent"
    if case["properties"]["Offense"] in non_violent_crime:
        crime_info_pd.loc[im, "Crime Type"] = "Non-Violent"
    
    # Append longitude value to "Lng" column
    crime_info_pd.loc[im, "LNG"] = case["geometry"]["coordinates"][0]

    # Append Latitude value to "Lat" column
    crime_info_pd.loc[im, "LAT"] = case["geometry"]["coordinates"][1]   
    
    # Convert date info. into datetime format and temporatorily assign it to "date_im"
    date_im = datetime.datetime.strptime(case["properties"]["Time_Begun"][:10], "%Y-%m-%d")
    # Append date info into "crime_date" list as "yyyy-mm-dd"
    crime_date.append(date_im.strftime("%Y-%m-%d"))
    
# Preview "crime_info_pd"
crime_info_pd.head()

,Offense,Crime Type,ZIP,LAT,LNG
0,Auto Theft,Non-Violet,77021,29.7112,-95.381
1,Auto Theft,Non-Violet,77546,29.5434,-95.1457
2,Theft,Non-Violet,77071,29.6442,-95.5083
3,Theft,Non-Violet,77036,29.6753,-95.5507
4,Auto Theft,Non-Violet,77075,29.652,-95.2637


In [10]:
# # Remove cases of crime with missing data if any
# crime_info_pd = crime_info_pd.dropna(how="any")

# Check the length of "crime_info_pd"
len(crime_info_pd)

7368

In [11]:
# Print the date range of HPD recent crime data 
date_range = str("from " + min(crime_date) + " to " + max(crime_date))

print("Crime Report" + " " + date_range)

Crime Report from 2019-02-21 to 2019-03-20


In [12]:
# Write "crime_info_pd" to csv file
crime_info_pd.to_csv("../data/cleandata/HPD_Crime_Report.csv", index=False, header=True)

In [13]:
# Group "crime_info_pd" by "Zip" and "Crime Type" and calculate violent and non-violent crime counts with given zipcodes
crime_ct = crime_info_pd.groupby(["ZIP", "Crime Type"]).count().reset_index()

# Preview "crime_ct"
crime_ct.head()

,ZIP,Crime Type,Offense,LAT,LNG
0,77002,Non-Violet,148,148,148
1,77002,Violet,32,32,32
2,77003,Non-Violet,54,54,54
3,77003,Violet,7,7,7
4,77004,Non-Violet,131,131,131


In [14]:
# Remove "LAT" and "LNG" columns
crime_ct = crime_ct.drop(["LAT", "LNG"], axis=1)

# Preview "crime_ct"
crime_ct.head()

,ZIP,Crime Type,Offense
0,77002,Non-Violet,148
1,77002,Violet,32
2,77003,Non-Violet,54
3,77003,Violet,7
4,77004,Non-Violet,131


In [15]:
# Add empty new columns "Non-Violent Ct" and "Violent Ct" to "crime_ct"
crime_ct["Non-Violent Ct"] = ""
crime_ct["Violent Ct"] = ""

# Preview "crime_ct"
crime_ct.head()

,ZIP,Crime Type,Offense,Non-Violet Ct,Violet Ct
0,77002,Non-Violet,148,,
1,77002,Violet,32,,
2,77003,Non-Violet,54,,
3,77003,Violet,7,,
4,77004,Non-Violet,131,,


In [16]:
# Save values in "Offense" to "Non-Violent Ct" and "Violent Ct" columns
# Write "0" for columns without data to facilitate following manipulation
crime_ct.loc[crime_ct["Crime Type"] == "Violent", "Violent Ct"] = crime_ct.loc[crime_ct["Crime Type"] == "Violent", "Offense"]
crime_ct.loc[crime_ct["Crime Type"] == "Violent", "Non-Violent Ct"] = 0
crime_ct.loc[crime_ct["Crime Type"] == "Non-Violent", "Non-Violent Ct"] = crime_ct.loc[crime_ct["Crime Type"] == "Non-Violent", "Offense"]
crime_ct.loc[crime_ct["Crime Type"] == "Non-Violent", "Violent Ct"] = 0

# Preview "crime_ct"
crime_ct.head()

,ZIP,Crime Type,Offense,Non-Violet Ct,Violet Ct
0,77002,Non-Violet,148,148,0
1,77002,Violet,32,0,32
2,77003,Non-Violet,54,54,0
3,77003,Violet,7,0,7
4,77004,Non-Violet,131,131,0


In [17]:
# Delete "Crime Type" and "Offense" columns from "crime_ct"
crime_info_ct = crime_ct.drop(["Crime Type", "Offense"], axis=1)

# Preview "crime_info_ct"
crime_info_ct.head()

,ZIP,Non-Violet Ct,Violet Ct
0,77002,148,0
1,77002,0,32
2,77003,54,0
3,77003,0,7
4,77004,131,0


In [18]:
# Merge data with the same zipcode
crime_info_ct = crime_info_ct.groupby("ZIP").sum().reset_index()

# Preview "crime_info_ct"
crime_info_ct.head()

,ZIP,Non-Violet Ct,Violet Ct
0,77002,148,32
1,77003,54,7
2,77004,131,42
3,77005,30,1
4,77006,131,16


In [19]:
len(crime_info_ct)

105

In [21]:
# Write "crime_info_pd" to csv file
crime_info_ct.to_csv("../data/cleandata/HPD_Crime_Report_Counts.csv", index=False, header=True)